In [24]:
import pandas as pd

from config import DATA_DIR
from utils.mongo_controller import mongo_controller

In [25]:
def extract_usdt_bob_binance_data(row):
    data = row['USDT_BOB_Binance']
    if isinstance(data, dict):
        return pd.Series({
            'USDT_BOB_Binance_sell_vwap': data.get('sell_vwap'),
            'USDT_BOB_Binance_buy_vwap': data.get('buy_vwap'),
            'USDT_BOB_Binance_sell_volume': data.get('sell_volume'),
            'USDT_BOB_Binance_buy_volume': data.get('buy_volume')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USDT_BOB_Binance_sell_vwap': None,
            'USDT_BOB_Binance_buy_vwap': None,
            'USDT_BOB_Binance_sell_volume': None,
            'USDT_BOB_Binance_buy_volume': None
        })

In [26]:
def extract_usd_bob_parallel_data(row):
    data = row['USD_BOB_Parallel']
    if isinstance(data, dict):
        return pd.Series({
            'USD_BOB_Parallel_sell_price': data.get('exchange_rate')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USD_BOB_Parallel_sell_price': None
        })

In [27]:
def extract_daily_usd_bob_parallel_curve(row):
    data = row.get('USD_BOB_Parallel_series', None)
    return pd.Series({'USD_BOB_Parallel_series': data})


def extract_other_usd_bob_parallel_curve(row):
    data = row.get('USD_BOB_Parallel', None)
    if data is not None and isinstance(data, dict):
        return pd.Series({'USD_BOB_Parallel_series': data.get('series_average')})
    else:
        return pd.Series({'USD_BOB_Parallel_series': None})


In [28]:
def extract_usd_bob_tarjeta_data(row):
    data = row['USD_BOB_Tarjeta']
    if isinstance(data, dict):
        return pd.Series({
            'USD_BOB_Tarjeta_sell_price': data.get('sell_price')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USD_BOB_Tarjeta_sell_price': None
        })

In [29]:
def extract_usdt_ars_binance_data(row):
    data = row['USDT_ARS_Binance']
    if isinstance(data, dict):
        return pd.Series({
            'USDT_ARS_Binance_sell_vwap': data.get('sell_vwap'),
            'USDT_ARS_Binance_buy_vwap': data.get('buy_vwap'),
            'USDT_ARS_Binance_sell_volume': data.get('sell_volume'),
            'USDT_ARS_Binance_buy_volume': data.get('buy_volume')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USDT_ARS_Binance_sell_vwap': None,
            'USDT_ARS_Binance_buy_vwap': None,
            'USDT_ARS_Binance_sell_volume': None,
            'USDT_ARS_Binance_buy_volume': None
        })

In [30]:
def extract_usd_ars_parallel_data(row):
    data = row['USD_ARS_Parallel']
    if isinstance(data, dict):
        return pd.Series({
            'USDT_ARS_Parallel_sell_price': data.get('sell_price')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USDT_ARS_Parallel_sell_price': None
        })

In [31]:
def extract_usdt_ars_tradingview_data(row):
    data = row['USDT_ARS_TradingView']
    if isinstance(data, dict):
        return pd.Series({
            'USDT_ARS_TradingView_open': data.get('open'),
            'USDT_ARS_TradingView_close': data.get('close'),
            'USDT_ARS_TradingView_high': data.get('high'),
            'USDT_ARS_TradingView_low': data.get('low'),
            'USDT_ARS_TradingView_volume': data.get('volume')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USDT_ARS_TradingView_open': None,
            'USDT_ARS_TradingView_close': None,
            'USDT_ARS_TradingView_high': None,
            'USDT_ARS_TradingView_low': None,
            'USDT_ARS_TradingView_volume': None
        })

In [32]:
def extract_usd_ars_official(row):
    data = row['USD_ARS_Official']
    if isinstance(data, dict):
        return pd.Series({
            'USD_ARS_Official_open': data.get('open'),
            'USD_ARS_Official_close': data.get('close'),
            'USD_ARS_Official_high': data.get('high'),
            'USD_ARS_Official_low': data.get('low')
        })
    else:
        # If data is None (or not a dict), return None for each new column
        return pd.Series({
            'USD_ARS_Official_open': None,
            'USD_ARS_Official_close': None,
            'USD_ARS_Official_high': None,
            'USD_ARS_Official_low': None
        })

In [36]:
for fiat_currency in ['ARS', 'BOB']:
    for data_mode in ['Quarterly', 'Monthly', 'Daily']:
        data_source_df = mongo_controller.query_data(_mode='all', collection=f'{data_mode}_Averages')
        data_source_df = data_source_df.drop(columns=['_id', 'USD_BOB_Official'])
        data_final_df = data_source_df.copy()

        if fiat_currency == 'BOB':
            expanded_columns = data_source_df.apply(extract_usdt_bob_binance_data, axis=1)
            data_final_df = data_final_df.join(expanded_columns)

            if data_mode != 'Daily':
                expanded_columns = data_source_df.apply(extract_other_usd_bob_parallel_curve, axis=1)
                data_final_df = data_final_df.join(expanded_columns)

            # expanded_columns = data_source_df.apply(extract_usd_bob_parallel_data, axis=1)
            # data_final_df = data_final_df.join(expanded_columns)

            expanded_columns = data_source_df.apply(extract_usd_bob_tarjeta_data, axis=1)
            data_final_df = data_final_df.join(expanded_columns)

            data_final_df.drop(columns=['USDT_BOB_Binance', 'USD_BOB_Parallel', 'USD_BOB_Tarjeta'], inplace=True)
            data_final_df.drop(
                columns=['USDT_ARS_Binance', 'USD_ARS_Parallel', 'USDT_ARS_TradingView', 'USD_ARS_Official'],
                inplace=True)

        if fiat_currency == 'ARS':
            # expanded_columns = data_source_df.apply(extract_usdt_ars_binance_data, axis=1)
            # data_final_df = data_final_df.join(expanded_columns)

            expanded_columns = data_source_df.apply(extract_usd_ars_parallel_data, axis=1)
            data_final_df = data_final_df.join(expanded_columns)

            expanded_columns = data_source_df.apply(extract_usdt_ars_tradingview_data, axis=1)
            data_final_df = data_final_df.join(expanded_columns)

            expanded_columns = data_source_df.apply(extract_usd_ars_official, axis=1)
            data_final_df = data_final_df.join(expanded_columns)

            data_final_df.drop(
                columns=['USDT_ARS_Binance', 'USD_ARS_Parallel', 'USDT_ARS_TradingView', 'USD_ARS_Official'],
                inplace=True)
            data_final_df.drop(columns=['USDT_BOB_Binance', 'USD_BOB_Parallel', 'USD_BOB_Tarjeta'], inplace=True)
            if data_mode == 'Daily':
                data_final_df.drop(columns=['USD_BOB_Parallel_series'], inplace=True)

        if data_mode == 'Daily':
            data_final_df = data_final_df.sort_values(by='timestamp', ascending=True)
        elif data_mode == 'Monthly':
            # Convert months that are int to str and add a leading zero if necessary
            data_final_df['month'] = data_final_df['month'].astype(str).str.zfill(2)
            data_final_df = data_final_df.sort_values(by=['year', 'month'], ascending=True)
        elif data_mode == 'Quarterly':
            data_final_df = data_final_df.sort_values(by=['year', 'quarter'], ascending=True)

        filename = DATA_DIR / f'{fiat_currency}_{data_mode.lower()}_data.csv'
        data_final_df.to_csv(filename, index=False)